# Project: Approximation of Option Premiums Using The Greeks
We know that the option premium is a function of several market conditions, such as the underlying share price, the duration left on the contract, etc. We can construct a local model of the change for a call contract using the expansion:
$$
\begin{equation}
d\mathcal{P}_{c} \sim \Delta_{c}\cdot{d}{S}+\frac{\Gamma_{c}}{2}\cdot\left(d{S}\right)^2 + \Theta_{c}\cdot{d}{T}+V_{c}\cdot{d}\sigma + \rho\cdot{d}{r}
\end{equation}
$$
where the values $\Delta_{c}, \Gamma_{c},  \Theta_{c}, V$ and $\rho$ denote [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) we computed previously. In the case study below, we estimate the change in the premium $d\mathcal{P}_{*}$, where $*\in\left\{c,p\right\}$ by computing the scalar product between the Greek values vector and the change vector:
$$
d\mathcal{P}_{*}\sim\left<\mathcal{G}_{*},\delta\right>\quad{*\in\left\{c,p\right\}}
$$
where $\left<\cdot,\cdot\right>$ denotes the [inner or scalar product](https://en.wikipedia.org/wiki/Inner_product_space) between the Greek value vector:
$$
\begin{equation}
\mathcal{G}_{*}\equiv\left(\Delta_{*},\Gamma_{*},\Theta_{*},V_{*},\rho_{*}\right)\quad{*\in\left\{c,p\right\}}
\end{equation}
$$
and the market perturbation vector $\delta$:
$$
\delta\equiv\left(dS,1/2\cdot{dS}^{2},dT,d\sigma,dr\right)
$$

## Learning objectives
This project will familiarize students with using [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) to compute how the price of `NVDA` option contracts change with market conditions, in particular the underlying share price $S$ and the implied volatility, denoted in this study by the symbol $\sigma$.

* __Prerequisites__: We'll begin by setting some parameters for the [CRR binomial lattice calculations for option premium](https://en.wikipedia.org/wiki/Binomial_options_pricing_model), data and models for the `NVDA` contracts, and finally we'll compute values for [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) for example long `NVDA` call and put contracts.
* __Task 1__: Compute changes in the premium for long `NVDA` call options.
    * `Check`: How good are the Greek `NVDA` call premium projections?
    * `Visualize` and discuss changes in the contract premium as a function of the underlying share price $S$ and the implied volatility $\sigma$.
* __Task 2__: Compute changes in the premium for long `NVDA` put options.
    * `Check`: How good are the Greek `NVDA` put premium projections?
    * `Visualize` and discuss changes in the contract premium as a function of the underlying share price $S$ and the implied volatility $\sigma$. 

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Prerequisites
Before we estimate how `call` and `put` option premiums change with market conditions using [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw), let's specify values for [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) and some other parameters for our test `call` and `put` options on [NVIDIA Corporation](https://finance.yahoo.com/quote/NVDA/).
* In the `prerequisites` section, we set parameters that we use in the [CRR binomial lattice calculations for option premium](https://en.wikipedia.org/wiki/Binomial_options_pricing_model), specify call and put contract models (instances of the [MyAmericanPutContractModel](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanPutContractModel) and [MyAmericanCallContractModel](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanCallContractModel) types) which hold data for the contracts, and compute values for [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) for the contracts using the[delta](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta), 
[theta](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta),
[vega](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega),
[rho](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta) and 
[gamma](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma) functions introduced in the worked example.

### Parameters and constants
Let's set a value for the step size `Δt,` i.e., `1 day` in units of years (assuming a `365-day` year), the risk-free rate at the time we gathered the data for the option contracts; we approximate this value using the [yield on the 10-year Treasury Note](https://ycharts.com/indicators/10_year_treasury_rate). We also set a value for the underlying share price `Sₒ` and the number of levels in the binomial tree model, `h`; we assume 6-levels per day.

In [210]:
Δt = (1/365);
risk_free_rate = 0.0431; # yield 10-year treasury note on 13-Jun-2024
Sₒ = 131.88; # after close on 14-Jun-2024 (close: 131.88)
h = 366; # 6-points per trading day
β = range(0.95,stop = 1.05, length=3) |> collect; # peturbation vector, values -5%, 0, +5%
α = range(0.80,stop = 1.20, length=3) |> collect; # peturbation vector, values -20%, 0, +20%

### Contract models
Next, we construct contract models for the options in this example. We begin by building an instance of the [MyAmericanCallContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanCallContractModel) using the [custom build(...) method for this type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}). The [build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}) takes two arguments: the type to build, and a [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) containing the strike price `K`, the `premium` paid for the option, the days to expiration `DTE` (units: years), the `sense = 1` (long), the number of contracts in the `copy` field and the implied volatility in the `IV` field.

We save the call option contract model in the `call_option_model` variable:

In [8]:
call_option_model = build(MyAmericanCallContractModel, (
    K = 136.0, 
    premium = 9.23, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4848
)); # contract variables: accessed before market open 2024-06-16

The put contract, which is an instance of the [MyAmericanPutContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanPutContractModel), is constructed in a similar way to the call option. We save the put option contract model in the `put_option_model` variable:

In [208]:
put_option_model = build(MyAmericanPutContractModel, (
    K = 128.0, 
    premium = 7.78, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4787
)); # contract variables: accessed before market open 2024-06-16

### Calculate the Greek values
Now that we have the contract models, we can compute [the Greek values](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) and store them in the `long_greek_dictionary.` The `long_greek_dictionary` variable is a [Julia Dictionary type](https://docs.julialang.org/en/v1/base/collections/#Dictionaries), where the key is the [Julia Union type](https://docs.julialang.org/en/v1/base/base/#Core.Union) between the contract model types, i.e.,  [MyAmericanPutContractModel](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanPutContractModel) $\cup$ [MyAmericanCallContractModel](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanCallContractModel) and the values are [NamedTuples](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) holding the various Greek values.
* __Union type, what the what?__ The [Julia Union type](https://docs.julialang.org/en/v1/base/base/#Core.Union) allows us to construct a composite type of sorts where we can use either a [MyAmericanPutContractModel](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanPutContractModel) __or__ a [MyAmericanCallContractModel](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanCallContractModel) as the key for the `long_greek_dictionary.` Wow, that is Nice!

In [12]:
long_greek_dictionary = Dict{Union{MyAmericanPutContractModel,MyAmericanCallContractModel}, NamedTuple}();

Next, compute and store the Greek values for the `call` option in the `long_greek_dictionary` key'd by the `call_option_model` instance. To do this, we call the [delta](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta), 
[theta](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta),
[vega](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega),
[rho](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta) and 
[gamma](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma) functions, with the appropriate contract model, i.e., the `call_option_model` variable for the call contract,  and the various lattice parameters: 

In [14]:
long_greek_dictionary[call_option_model] = (
    delta_value = delta(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    theta_value = theta(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    vega_value = vega(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    rho_value = rho(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    gamma_value = gamma(call_option_model, h = h,  T = call_option_model.DTE, 
        σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate));

Similarly, we compute and store the Greek values for the `put` option in the `long_greek_dictionary` key'd in this case by the `put_option_model` instance, and using the `put_option_model` instance in the various functions:

In [235]:
long_greek_dictionary[put_option_model] = (
    delta_value = delta(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    theta_value = theta(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    vega_value = vega(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    rho_value = rho(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate),
    gamma_value = gamma(put_option_model, h = h,  T = put_option_model.DTE, 
        σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate));

## Task 1: Compute changes in the premium for long call options
Let's use [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) to explore how the price of an `NVDA` long call contract changes with changes in the underlying share price and the implied volatility of the contract. First, access the `long_greek_dictionary,` pull out the Greek values for this contract from the `call_greek_tuple` [NamedTuple data structure](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple), and manually put them in the `call_greek_vector` variable. 
* __Why this order__? We choose an arbitrary order for the Greek values according to the market variable we are interested in: the underlying share price, the duration left on the contract, the implied volatility, and the risk-free rate. There is nothing magical about this order. However, below, when we compute the $\delta$-vector, the variables must be in this order. 

In [18]:
call_greek_tuple = long_greek_dictionary[call_option_model];
call_greek_vector = [
    call_greek_tuple.delta_value,
    call_greek_tuple.gamma_value,
    call_greek_tuple.theta_value,
    call_greek_tuple.vega_value,
    call_greek_tuple.rho_value
];

__Main simulation loop__: In the code block below, we calculate the scaled change in the call option premium, denoted as $\Delta\mathcal{P}/\mathcal{P}_{\circ}$, resulting from different underlying share prices and implied volatility values using nested `for-loops.`
* First, we initialize the `long_call_simulation_array::Array{Float64,2}` array which holds the $\Delta\mathcal{P}/\mathcal{P}_{\circ}$ values. We then define the `IVₒ` and `Pₒ` variables, which hold the default implied volatility and option premium values.
* In the outer loop with index `i`, we select a perturbation value $\beta_{i}\in\beta$ for the share price $S$, while in the inner loop with index `j,` we select a perturbation value $\alpha_{j}\in\alpha$ for the implied volatility.
* On lines `8-10`, we compute the `δ_vector` and then calculate the scalar product between the `call_greek_vector` and the `δ-vector` using the [Julia dot(...) function exported by LinearAlgebra.jl](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.dot).

In [20]:
long_call_simulation_array = Array{Float64,2}(undef,length(β),length(α));
IVₒ = call_option_model.IV;
Pₒ = call_option_model.premium
for i ∈ eachindex(β)
    β_outer = β[i];
    for j ∈ eachindex(α)
        α_inner = α[j];         
        δ_vector = [
            (β_outer - 1)*Sₒ, (1/2)*((β_outer - 1)*Sₒ)^2, 0, 100*(α_inner - 1)*IVₒ, 0 # Hmm. What is going on here?
        ];
        long_call_simulation_array[i,j] = dot(call_greek_vector, δ_vector)/Pₒ
    end
end

### Check: How good are the Greek call premium projections?
The Greeks provide a tool to estimate the premium of an options contract based on market conditions, such as changes in implied volatility or the underlying stock price. However, the accuracy of these approximations is unknown.

 * __Test 1__: Calculate the premium change for the long `NVDA` call option when there is a `+20%` change in implied volatility (and the nominal underlying share price), using the binomial options pricing model. After obtaining the results, find the percentage difference between the Greek and binomial calculations. Finally, verify if the two values differ by less than $\pm$ `5%` using the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert).

 * __Test 2__: Calculate the change in the premium of the long `NVDA` call contract for a `-5%` change in the underlying share price, using the binomial options pricing model with a nominal value for the implied volatility. After that, find the percentage difference between the Greek and binomial calculations, and then check if the two values differ by less than $\pm$ `5%`, using the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert)

`Unhide` the code below to see how we performed __Test 1__.

In [24]:
let
    σₒ = call_option_model.IV; # nominal IV
    σ₁ = σₒ*1.2; # 20% increase in the IV

    # tree models -
    mₒ = build(MyAdjacencyBasedCRREquityPriceTree, 
        (μ = risk_free_rate, T = call_option_model.DTE, σ = σₒ)) |> (x-> populate(x, Sₒ = Sₒ, h = h)); # nominal

    m₁ = build(MyAdjacencyBasedCRREquityPriceTree, 
        (μ = risk_free_rate, T = call_option_model.DTE, σ = σ₁)) |> (x-> populate(x, Sₒ = Sₒ, h = h)); # perturbed

    # premium -
    Pₒ = premium(call_option_model, mₒ); # nominal premium
    P₁ = premium(call_option_model, m₁); # perturbed premium

    # calculate the diff -
    diff_binomial = (P₁ - Pₒ)/Pₒ;
    diff_greek = long_call_simulation_array[2,3];

    # report -
    @assert abs((diff_binomial - diff_greek)) ≤ 0.05; # less than a 5% error?
end

`Unhide` the code below to see how we performed __Test 2__.

In [26]:
let

    # underlying price
    S₁ = 0.95*Sₒ 
    
    # tree models -
    mₒ = build(MyAdjacencyBasedCRREquityPriceTree, 
        (μ = risk_free_rate, T = call_option_model.DTE, σ = call_option_model.IV)) |> (x-> populate(x, Sₒ = Sₒ, h = h)); # nominal

    m₁ = build(MyAdjacencyBasedCRREquityPriceTree, 
        (μ = risk_free_rate, T = call_option_model.DTE, σ = call_option_model.IV)) |> (x-> populate(x, Sₒ = S₁, h = h)); # perturbed

    # premium -
    Pₒ = premium(call_option_model, mₒ); # nominal premium
    P₁ = premium(call_option_model, m₁); # perturbed premium

    # calculate the diff -
    diff_binomial = (P₁ - Pₒ)/Pₒ;
    diff_greek = long_call_simulation_array[1,2];

    # report -
    @assert abs((diff_binomial - diff_greek)) ≤ 0.05; # less than a 5% error?
end

### Visualize
`Unhide` the code block below to see how we constructed the table holding [the Greek](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) premium calculations using the [the pretty_table(...) function exported by PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl).
* __Summary share price $S$__: When you hold a long call option, it's similar to owning the underlying shares. The change in the option price as a function of the change in the underlying share price can be approximated as:
$$
\begin{equation}
d\mathcal{P}_{c}\sim\Delta_{c}\cdot{d}{S}+\frac{\Gamma_{c}}{2}\cdot\left(d{S}\right)^2
\end{equation}
$$
assuming all other factors remain constant. Since $\Delta_{c}>0$ and $\Gamma_{c}>0$ for the long `NVDA` call contract, the option price moves in the same direction as changes in the underlying share price $\partial{S}$. Thus, the long `NVDA` call makes a `bullish` directional assumption.
* __Summary implied volatility IV__: When you hold a long call option, you are hoping for the option's price to increase so that you can sell it for more than you paid for it before it expires. Assuming there are no changes in other market variables, the change in the option price due to a change in implied volatility (represented by the symbol $\sigma$ in the Greek expansion) can be approximated as:
$$
\begin{equation}
d\mathcal{P}_{c}\sim{V}_{c}\cdot{d}{\sigma}
\end{equation}
$$
For a long `NVDA` call, since $V_{c}>0$, an increase in implied volatility ($d\sigma>0$) leads to an increase in the option value, while a decrease in implied volatility ($d\sigma<0$) reduces the price of the long `NVDA` call contract.

In [22]:
let
    (R,C) = size(long_call_simulation_array)
    pretty_table_data = Array{Any,2}(undef, R, C+1)
    
    # first col holds labels -
    for i ∈ 1:R
        if (i == 1)
            pretty_table_data[i,1] = "-5% S";
        elseif (i == 3)
            pretty_table_data[i,1] = "+5% S";
        else
            pretty_table_data[i,1] = "nominal S";
        end
    end
    
    for i = 1:R
        for j = 1:C
            pretty_table_data[i,j+1] = long_call_simulation_array[i,j]
        end
    end
    
    header_data = (["", "-20% IV", "nominal IV", "+20% IV"])
    pretty_table(pretty_table_data, header=header_data)
end

┌───────────┬───────────┬────────────┬────────────┐
│           │   -20% IV │ nominal IV │    +20% IV │
├───────────┼───────────┼────────────┼────────────┤
│     -5% S │  -0.52422 │  -0.297315 │ -0.0704098 │
│ nominal S │ -0.226905 │        0.0 │   0.226905 │
│     +5% S │  0.178759 │   0.405664 │   0.632569 │
└───────────┴───────────┴────────────┴────────────┘


## Task 2: Compute changes in the premium for long-put options
Following the procedure for the call option above, we begin by populating the `put_greek_vector` variable by accessing the `put_greek_tuple` [NamedTuple data structure](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) from the `long_greek_dictionary.`  The Greek values are in the same order as the $\delta$-vector:

In [173]:
put_greek_tuple = long_greek_dictionary[put_option_model]; # we pass the put_option_model for put data
put_greek_vector = [
    put_greek_tuple.delta_value,
    put_greek_tuple.gamma_value,
    put_greek_tuple.theta_value,
    put_greek_tuple.vega_value,
    put_greek_tuple.rho_value
];

__Main simulation loop__: In the code block below, we calculate the scaled change in the put option premium, denoted as $\Delta\mathcal{P}/\mathcal{P}_{\circ}$, resulting from different underlying share prices and implied volatility values using nested `for-loops.`
* This logic follows the same ideas as the call option case shown above, with the exception that we are using data for the `NVDA` put option and storing data in the `long_put_simulation_array::Array{Float64,2}` variable:

In [46]:
long_put_simulation_array = Array{Float64,2}(undef,length(β),length(α));
IVₒ = put_option_model.IV;
Pₒ = put_option_model.premium
for i ∈ eachindex(β)
    β_outer = β[i];
    for j ∈ eachindex(α)
        α_inner = α[j];         
        δ_vector = [
            (β_outer - 1)*Sₒ, (1/2)*((β_outer - 1)*Sₒ)^2, 0, 100*(α_inner - 1)*IVₒ, 0 # do you understand this?
        ];
        long_put_simulation_array[i,j] = dot(put_greek_vector, δ_vector)/Pₒ
    end
end

### Check: How good are the Greek put premium projections?
Write tests to assess the accuracy of [the Greek]() premium approximation for long `NVDA` put options.

 * __Test 1__: Calculate the premium change for the long `NVDA` call option when there is a `+20%` change in implied volatility (and the nominal underlying share price), using the binomial options pricing model. After obtaining the results, find the percentage difference between the Greek and binomial calculations. Finally, verify if the two values differ by less than $\pm$ `5%` using the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert).

 * __Test 2__: Calculate the change in the premium of the long `NVDA` call contract for a `-5%` change in the underlying share price, using the binomial options pricing model with a nominal value for the implied volatility. After that, find the percentage difference between the Greek and binomial calculations, and then check if the two values differ by less than $\pm$ `5%`, using the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert)

`Enter` the code below to perform __Test 1__ for the `NVDA` put option:

In [270]:
let
    # Students: input Test 1 logic here for put; see call example above
end

`Enter` the code below to perform __Test 2__ for the `NVDA` put option:

In [268]:
let
    # Students: Input Test 2 logic here for put; see call example above
end

### Visualize
`Unhide` the code block below to see how we constructed the table holding [the Greek](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) premium calculations using the [the pretty_table(...) function exported by PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl)
* __Summary share price $S$__: When you hold a long put option, it means you are effectively shorting the underlying shares. Similar to a call option, the change in the option price in response to a change in the underlying share price can be roughly estimated by the following equation:
$$
\begin{equation}
d\mathcal{P}_{p}\sim\Delta_{p}\cdot{d}{S}+\frac{\Gamma_{p}}{2}\cdot\left(d{S}\right)^2
\end{equation}
$$
assuming all other factors remain constant. Since $\Delta_{p}<0$ and $\Gamma_{p}>0$, but $|\Delta_{p}|\gg|\Gamma_{p}|$ for the long `NVDA` put contract, it means that the option price moves in the opposite direction of the underlying share price $\partial{S}$, and the long `NVDA` put reflects a bearish directional assumption.
* __Summary implied volatility IV__: When you hold a long call option, you are hoping for the option's price to increase so that you can sell it for more than you paid for it before it expires. Assuming there are no changes in other market variables, the change in the option price due to a change in implied volatility (represented by the symbol $\sigma$ in the Greek expansion) can be approximated as:
$$
\begin{equation}
d\mathcal{P}_{p}\sim{V}_{p}\cdot{d}{\sigma}
\end{equation}
$$
For a long `NVDA` call, since $V_{c}>0$, an increase in implied volatility ($\partial\sigma>0$) leads to an increase in the option value, while a decrease in implied volatility ($\partial\sigma<0$) reduces the price of the long `NVDA` call contract.

In [51]:
let
    (R,C) = size(long_put_simulation_array)
    pretty_table_data = Array{Any,2}(undef, R, C+1)
    
    # first col holds labels -
    for i ∈ 1:R
        if (i == 1)
            pretty_table_data[i,1] = "-5% S";
        elseif (i == 3)
            pretty_table_data[i,1] = "+5% S";
        else
            pretty_table_data[i,1] = "nominal S";
        end
    end
    
    for i = 1:R
        for j = 1:C
            pretty_table_data[i,j+1] = long_put_simulation_array[i,j]
        end
    end
    
    header_data = (["", "-20% IV", "nominal IV", "+20% IV"])
    pretty_table(pretty_table_data, header=header_data)
end

┌───────────┬───────────┬────────────┬────────────┐
│           │   -20% IV │ nominal IV │    +20% IV │
├───────────┼───────────┼────────────┼────────────┤
│     -5% S │  0.130008 │    0.38474 │   0.639473 │
│ nominal S │ -0.254732 │        0.0 │   0.254732 │
│     +5% S │ -0.527697 │  -0.272964 │ -0.0182321 │
└───────────┴───────────┴────────────┴────────────┘


## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.